1. Read *Wikipedia.csv* as written via Wikipedia class.
2. Tokenize the text, and save tokens into *df_token*.
3. Build similarity matrix and write this to file *Wikipedia-sims.csv* with columns:  
    id : *article id*  
    similar_articles : *list of article ids*


In [1]:
import sys, os
import pandas as pd

# Read data

Read .csv files and make a pandas dataframe.

In [2]:
actual_dir=os.getcwd()
os.chdir('../data')
csvfile1 = 'Wikipedia-dog.csv'
csvfile2 = 'Wikipedia-fish.csv'

df=pd.read_csv(csvfile1,index_col=0)
df2=pd.read_csv(csvfile2,index_col=0)
df=df.append(df2)
del df2

df=df[df['page_type']=='article']


del df['page_type']
del df['text_raw']

df.columns=['url','title']

df.tail(5)




,url,title
43590845,https://en.wikipedia.org/wiki/Whale_feces,Whale feces
42614454,https://en.wikipedia.org/wiki/Whale_watching_i...,Whale watching in New Zealand
49735416,https://en.wikipedia.org/wiki/Tail_sailing,Tail sailing
52243894,https://en.wikipedia.org/wiki/Bubble_net_feeding,Bubble net feeding
53720250,https://en.wikipedia.org/wiki/Ethelbert_(whale),Ethelbert (whale)


In [3]:

model='LDA'


if ( model == 'LSI' ):
    df_sims=pd.read_csv('Wikipedia-sims-lsi.csv',index_col=0)

elif (model == 'LDA' ):
    df_sims=pd.read_csv('Wikipedia-sims-lda.csv',index_col=0)

df_sims.head()

,similar_articles,scores
1467938,"[54435831, 236573, 33069847, 66365, 47048387, ...","[0.99430513381958008, 0.33144289255142212, 0.3..."
275388,"[50714180, 44137874, 51676708, 10579085, 52014...","[0.95121949911117554, 0.95121949911117554, 0.9..."
2352562,"[4269567, 76940, 8526165, 17862013, 9369613, 5...","[0.99589186906814575, 0.98597866296768188, 0.9..."
17021807,[],[]
20777185,"[44444481, 5205615, 53903332, 17862013, 539202...","[0.82681477069854736, 0.79914122819900513, 0.6..."


In [4]:
from ast import literal_eval
import numpy as np

# Put titles into a dataframe
df_titles=df['title']
df_urls=df['url']

# Add empty columns to Pandas dataframe:
df['other titles']=""
df['other urls']=""
df['scores']=""

# Change similar_articles from ids, to titles

for article_id in df.index:
    other_articles_ids=df_sims.loc[article_id]['similar_articles']
    other_articles_scores=df_sims.loc[article_id]['scores']
    # Take only top 5 article ids and scores
    other_articles_ids=literal_eval(other_articles_ids)[:5]
    other_articles_scores=literal_eval(other_articles_scores)[:5]
    # Round scores, and convert numpy array to list
    other_articles_scores=np.round(other_articles_scores,2).tolist()
    # Select titles in other_articles_ids, and convert to list
    other_titles=df_titles.loc[other_articles_ids].values.tolist()
    # Select urls in other_articles_ids, and convert to list
    other_urls=df_urls.loc[other_articles_ids].values.tolist()
    
    # Fill DF entries:
    df['other titles'][article_id]=other_titles
    df['scores'][article_id]=other_articles_scores
    df['other urls'][article_id]=other_urls
df.head()

,url,title,other titles,other urls,scores
1467938,https://en.wikipedia.org/wiki/Kennel,Kennel,"[Pet Fed India, Dog food, Cropping (animal), K...","[https://en.wikipedia.org/wiki/Pet_Fed_India, ...","[0.99, 0.33, 0.33, 0.33, 0.33]"
275388,https://en.wikipedia.org/wiki/Cynology,Cynology,"[Dog Parker, Big Miracle (book), Indian Nation...","[https://en.wikipedia.org/wiki/Dog_Parker, htt...","[0.95, 0.95, 0.95, 0.95, 0.95]"
2352562,https://en.wikipedia.org/wiki/Pack_(canine),Pack (canine),"[Dog, Service dog, Cat senses, Dogs in religio...","[https://en.wikipedia.org/wiki/Dog, https://en...","[1.0, 0.99, 0.98, 0.98, 0.98]"
17021807,https://en.wikipedia.org/wiki/Rare_breed_(dog),Rare breed (dog),[],[],[]
20777185,https://en.wikipedia.org/wiki/Dogs_in_ancient_...,Dogs in ancient China,"[Becerrillo, Puppy, Body language of dogs, Dog...","[https://en.wikipedia.org/wiki/Becerrillo, htt...","[0.83, 0.8, 0.61, 0.6, 0.6]"


In [5]:
if ( model == 'LSI' ):
    df.to_csv('Wikipedia-db-lsi.csv')
elif ( model == 'LDA' ):
    df.to_csv('Wikipedia-db-lda.csv')
